This program implements a fair solution of readers and writers in Python using threads and semaphores. The program starts with one reader and one writer. The writer generates in 1 second intervals from 0 .. `numIters` (command line argument), starting from the middle and going up. The writer is given preference, so if there are multiple readers, all readers get updates from single writers. The program supports multiple readers and multiple writers.

In [9]:
from threading import Thread, Semaphore
from time import sleep
from sys import stdout

# global variables and semaphores
e = Semaphore(1)
r, w = Semaphore(0), Semaphore(0)
nr, nw = 0, 0
dr, dw = 0, 0
data = 0
maxdata = 0

def reader():
    global nr, nw, dr, dw, data, maxdata
    maxdata = 0
    stdout.write('Reader starting\n')
    while True:
        # entry protocol
        e.acquire()
        if nw > 0 or dw > 0:
            dr += 1; e.release(); r.acquire()
        nr += 1
        if dr > 0:
            dr -= 1; r.release()
        else:
            e.release()
        
        if data > maxdata:
            stdout.write('Reader in critical section read ' + str(data) + '\n')
        maxdata = data if data > maxdata else maxdata
        
        # exit protocol
        e.acquire()
        nr -= 1
        if nr == 0 and dw > 0:
            dw -= 1; w.release()
        else:
            e.release()

def writer(numIters):
    global nr, nw, dr, dw, data
    stdout.write('Writer starting\n')
    for i in range(numIters):
        # entry protocol
        e.acquire()
        if nr > 0 or nw > 0:
            dw += 1; e.release(); w.acquire()
        nw += 1
        e.release()
        
        stdout.write('Writer in critical section\n')
        data = (i + numIters // 2) % numIters
        stdout.write('Writer writing ' + str(data) + '\n')
        
        # exit protocol
        e.acquire()
        nw -= 1
        if dw > 0:
            dw -= 1; w.release()
        elif dr > 0:
            dr -= 1; r.release()
        else:
            e.release()
        
        sleep(1)

def rw(numIters):
    r = Thread(target = reader, daemon = True)
    w = Thread(target = writer, args = (numIters, ))
    r.start(); w.start()
    w.join()
    print("Max data " + str(maxdata))

In [10]:
rw(10)

Reader starting
Writer starting
Writer in critical section
Writer writing 5
Reader in critical section read 5
Writer in critical section
Writer writing 6
Reader in critical section read 6
Writer in critical section
Writer writing 7
Reader in critical section read 7
Writer in critical section
Writer writing 8
Reader in critical section read 8
Reader in critical section read 8
Reader in critical section read 8
Reader in critical section read 8
Reader in critical section read 8
Writer in critical section
Writer writing 9
Reader in critical section read 9
Reader in critical section read 9
Reader in critical section read 9
Reader in critical section read 9
Writer in critical section
Writer writing 0
Writer in critical section
Writer writing 1
Writer in critical section
Writer writing 2
Writer in critical section
Writer writing 3
Writer in critical section
Writer writing 4
Max data 9
